# DATA CLEANING AND TRANSFORMATION


In [74]:
#Libraries used for the exploratory analylis of the data

import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

%matplotlib inline

Opening the file of the Air indexes and the file of the weather metrics to verify the completness and correctness of the data

## AIR QUALITY INDEX REFERENCE FILE


Data is gather in real time by different stations locations across several areas in Madrid city.
This is storage in different format files. The study will use ".csv", which follow this data structure

PROVINCIA| MUNICIPIO | ESTACION | MAGNITUD | PUNTO DE MUESTREO | AÑO | MES | DIA | DATO | CODIGO DE VALIDACION
:--------: | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- 
28 | 079 | 004 | 01 | 28079004_1_38| 17 |07 | 01| 00005 | V


**PROVINCIA.** Spanish province where the study is located. 28 belongs to Madrid

**MUNICIPIO.** Municipality where the study is located. 079 belongs to Madrid city

**ESTACION.** Measuring station. In Madrid there are 24 stations across all the districts.

**MAGNITUD.** Every station measure different parameters related to air quality. __This study will only need Magnitud related to Monoxido de Nitrogeno, NO2 that is represented as Magnitud = "08".__

**PUNTO DE MUESTREO.** This is a code including the station id (including provincia, municipio y estacion), and adding the "magnitud" and "tecnica de muestreo"

**CODIGO DE VALIDACION.** Flag to identify the valid measurements, filtering out others that may have some issue during the measurement process. So, __the only valid values have a "V" in this field.__

**H01/VO1, H02/V02.** There are 48 extra columns related to hour (H01 = 1:00 am, H02 = 2:00 am..) and validation (V01 = data flag related to the data aat 1:00 am...). NOTE. For practical reason in the analyis the representation of the hours have been modified 1 second, just to allow reference the H24 to the same date. This is just matter of data representation not altering any result in the analysis.

In [71]:
#Creating dataframe with all data from the reference AirQuality file.

air_index = pd.read_csv ("../data/interim/AirQuality/ref_air_quality.csv", sep= ',')

In [72]:
air_index.head()


,Unnamed: 0,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,...,V20,H21,V21,H22,V22,H23,V23,H24,V24,source_file
0,0,28,79,4,1,28079004_1_38,2019,4,1,4.0,...,V,2.0,V,4.0,V,4.0,V,4.0,V,abr_mo19.csv
1,1,28,79,4,1,28079004_1_38,2019,4,2,3.0,...,V,4.0,V,4.0,V,4.0,V,4.0,V,abr_mo19.csv
2,2,28,79,4,1,28079004_1_38,2019,4,3,3.0,...,V,5.0,V,5.0,V,5.0,V,4.0,V,abr_mo19.csv
3,3,28,79,4,1,28079004_1_38,2019,4,4,4.0,...,V,4.0,V,4.0,V,3.0,V,4.0,V,abr_mo19.csv
4,4,28,79,4,1,28079004_1_38,2019,4,5,4.0,...,V,2.0,V,2.0,V,1.0,V,1.0,V,abr_mo19.csv


Checking null values in the different fields and rows. 

In [73]:
air_index.isnull().sum()

Unnamed: 0        0
PROVINCIA         0
MUNICIPIO         0
ESTACION          0
MAGNITUD          0
PUNTO_MUESTREO    0
ANO               0
MES               0
DIA               0
H01               0
V01               0
H02               0
V02               0
H03               0
V03               0
H04               0
V04               0
H05               0
V05               0
H06               0
V06               0
H07               0
V07               0
H08               0
V08               0
H09               0
V09               0
H10               0
V10               0
H11               0
V11               0
H12               0
V12               0
H13               0
V13               0
H14               0
V14               0
H15               0
V15               0
H16               0
V16               0
H17               0
V17               0
H18               0
V18               0
H19               0
V19               0
H20               0
V20               0
H21               0


**CONCLUSION:** No NaNs in the data

### CREATING POC DATASET


The analysis will start with a MVP analysis for 1 station just as a Probe of concept for the model, after considering that model is working reasonable well for 1 station the study will be extended to full station network.

The scope of the study is only based on NO2 index, so the values related to ther indexes in the field "MAGNITUD" should be ignored (not only in the POC, from full study)

The station #4 will be used for this POC

In [75]:
#Creating dataframe for the POC station and related to NO2 index

poc_air_index8_4 = air_index[(air_index['MAGNITUD'] == 8) & (air_index['ESTACION'] == 4)]

In [76]:
poc_air_index8_4

,Unnamed: 0,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,...,V20,H21,V21,H22,V22,H23,V23,H24,V24,source_file
90,90,28,79,4,8,28079004_8_8,2019,4,1,17.0,...,V,51.0,V,81.0,V,98.0,V,48.0,V,abr_mo19.csv
91,91,28,79,4,8,28079004_8_8,2019,4,2,22.0,...,V,60.0,V,45.0,V,50.0,V,45.0,V,abr_mo19.csv
92,92,28,79,4,8,28079004_8_8,2019,4,3,33.0,...,V,47.0,V,52.0,V,40.0,V,27.0,V,abr_mo19.csv
93,93,28,79,4,8,28079004_8_8,2019,4,4,20.0,...,V,34.0,V,34.0,V,32.0,V,23.0,V,abr_mo19.csv
94,94,28,79,4,8,28079004_8_8,2019,4,5,22.0,...,V,37.0,V,60.0,V,69.0,V,21.0,V,abr_mo19.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139169,139169,28,79,4,8,28079004_8_8,2021,6,26,33.0,...,V,10.0,V,12.0,V,18.0,V,23.0,V,jun_mo21.csv
139170,139170,28,79,4,8,28079004_8_8,2021,6,27,23.0,...,V,8.0,V,8.0,V,12.0,V,10.0,V,jun_mo21.csv
139171,139171,28,79,4,8,28079004_8_8,2021,6,28,9.0,...,V,10.0,V,13.0,V,16.0,V,18.0,V,jun_mo21.csv
139172,139172,28,79,4,8,28079004_8_8,2021,6,29,19.0,...,V,11.0,V,22.0,V,48.0,V,42.0,V,jun_mo21.csv


In [77]:
poc_air_index8_4.shape

(893, 58)

"Vxx" fields can contain N, what means that value related to that hour is not valid. So, as part of the cleaning and data transformation a deeper analysis will be needed to take a decision about the relevance of those values and impact in the final predicion, just to decide to clean this data from all stations or not.

In [70]:
#Checking if there are not valid values (with N in Vxx fields)

poc_air_index8_4['V01'].value_counts().to_frame()

,V01
V,870
N,23


**CONCLUSION.** The amount of Not valid measurement in a specific hour is not too relevant to the overall exercise

The current data structure is not optimal for the analyis, so different transformation steps are going to be done.

1. Creation of TimeStamp just looking to simplify the data structure to something better to read. This is done with the function "to_datetime", concatenating the 3 fields in the original data in a date.https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [78]:
# Creation of a "DATE" field with the concatenation of ANO, MES & DIA
poc_air_index8_4 ['DATE'] = pd.to_datetime({'year': poc_air_index8_4['ANO'],
                                                          'month': poc_air_index8_4['MES'],
                                                          'day': poc_air_index8_4['DIA']})

<ipython-input-78-522bc75d48ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poc_air_index8_4 ['DATE'] = pd.to_datetime({'year': poc_air_index8_4['ANO'],


In [79]:
poc_air_index8_4


,Unnamed: 0,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,...,H21,V21,H22,V22,H23,V23,H24,V24,source_file,DATE
90,90,28,79,4,8,28079004_8_8,2019,4,1,17.0,...,51.0,V,81.0,V,98.0,V,48.0,V,abr_mo19.csv,2019-04-01
91,91,28,79,4,8,28079004_8_8,2019,4,2,22.0,...,60.0,V,45.0,V,50.0,V,45.0,V,abr_mo19.csv,2019-04-02
92,92,28,79,4,8,28079004_8_8,2019,4,3,33.0,...,47.0,V,52.0,V,40.0,V,27.0,V,abr_mo19.csv,2019-04-03
93,93,28,79,4,8,28079004_8_8,2019,4,4,20.0,...,34.0,V,34.0,V,32.0,V,23.0,V,abr_mo19.csv,2019-04-04
94,94,28,79,4,8,28079004_8_8,2019,4,5,22.0,...,37.0,V,60.0,V,69.0,V,21.0,V,abr_mo19.csv,2019-04-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139169,139169,28,79,4,8,28079004_8_8,2021,6,26,33.0,...,10.0,V,12.0,V,18.0,V,23.0,V,jun_mo21.csv,2021-06-26
139170,139170,28,79,4,8,28079004_8_8,2021,6,27,23.0,...,8.0,V,8.0,V,12.0,V,10.0,V,jun_mo21.csv,2021-06-27
139171,139171,28,79,4,8,28079004_8_8,2021,6,28,9.0,...,10.0,V,13.0,V,16.0,V,18.0,V,jun_mo21.csv,2021-06-28
139172,139172,28,79,4,8,28079004_8_8,2021,6,29,19.0,...,11.0,V,22.0,V,48.0,V,42.0,V,jun_mo21.csv,2021-06-29


2. Hour information is storage in multiple columns, one per hour. This is not practical to manage the dataset, so a second transformation step has been done using "wide to long" concept, using "melt" function from pandas. This function is useful to massage a DataFrame into a format where one or more columns are identifier variables (id_vars), while all other columns, considered measured variables (value_vars), are “unpivoted” to the row axis, leaving just two non-identifier columns, ‘variable’ and ‘value’

https://pandas.pydata.org/docs/reference/api/pandas.melt.html#pandas.melt


In [80]:
# Dataframe transformation, keeping needed data and creating measured variables originally in columns

poc_air_index8_4 = pd.melt(poc_air_index8_4, id_vars=['ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO','DATE'], value_vars=['H01', 'H02', 'H03', 'H04', 'H05',
                                                                                                'H06', 'H07', 'H08', 'H09', 'H10',
                                                                                                'H11', 'H12', 'H13', 'H14', 'H15',
                                                                                                'H16', 'H17', 'H18', 'H19', 'H20',
                                                                                                'H21', 'H22', 'H23', 'H24'])

In [82]:
poc_air_index8_4

,ESTACION,MAGNITUD,PUNTO_MUESTREO,DATE,variable,value
0,4,8,28079004_8_8,2019-04-01,H01,17.0
1,4,8,28079004_8_8,2019-04-02,H01,22.0
2,4,8,28079004_8_8,2019-04-03,H01,33.0
3,4,8,28079004_8_8,2019-04-04,H01,20.0
4,4,8,28079004_8_8,2019-04-05,H01,22.0
...,...,...,...,...,...,...
21427,4,8,28079004_8_8,2021-06-26,H24,23.0
21428,4,8,28079004_8_8,2021-06-27,H24,10.0
21429,4,8,28079004_8_8,2021-06-28,H24,18.0
21430,4,8,28079004_8_8,2021-06-29,H24,42.0


3. Stil data require some other transformation to allow us manipulate and visualize it properly, it requires to transform the "variables" from columns created with melt function together with the date to create a Timestamp field, what will facilitate us the data manipulation.

In [88]:
#Creation of HORA field based on the hourly columns manipulated with melt funcion and now storage in "variable" field

poc_air_index8_4['HORA'] = [int(x[1:3]) for x in poc_air_index8_4.variable.values]

In [89]:
poc_air_index8_4

,ESTACION,MAGNITUD,PUNTO_MUESTREO,DATE,variable,value,HORA
0,4,8,28079004_8_8,2019-04-01,H01,17.0,1
1,4,8,28079004_8_8,2019-04-02,H01,22.0,1
2,4,8,28079004_8_8,2019-04-03,H01,33.0,1
3,4,8,28079004_8_8,2019-04-04,H01,20.0,1
4,4,8,28079004_8_8,2019-04-05,H01,22.0,1
...,...,...,...,...,...,...,...
21427,4,8,28079004_8_8,2021-06-26,H24,23.0,24
21428,4,8,28079004_8_8,2021-06-27,H24,10.0,24
21429,4,8,28079004_8_8,2021-06-28,H24,18.0,24
21430,4,8,28079004_8_8,2021-06-29,H24,42.0,24


In [94]:
#Creation of timestamp field

poc_air_index8_4['TIME'] = poc_air_index8_4['DATE'].astype(str) + ' ' + (poc_air_index8_4['HORA']-1).astype(str) + ':59:59'

In [96]:
poc_air_index8_4

,ESTACION,MAGNITUD,PUNTO_MUESTREO,DATE,variable,value,HORA,TIME,TIMESTAMP
0,4,8,28079004_8_8,2019-04-01,H01,17.0,1,2019-04-01 0:59:59,2019-04-01 0:59:59
1,4,8,28079004_8_8,2019-04-02,H01,22.0,1,2019-04-02 0:59:59,2019-04-02 0:59:59
2,4,8,28079004_8_8,2019-04-03,H01,33.0,1,2019-04-03 0:59:59,2019-04-03 0:59:59
3,4,8,28079004_8_8,2019-04-04,H01,20.0,1,2019-04-04 0:59:59,2019-04-04 0:59:59
4,4,8,28079004_8_8,2019-04-05,H01,22.0,1,2019-04-05 0:59:59,2019-04-05 0:59:59
...,...,...,...,...,...,...,...,...,...
21427,4,8,28079004_8_8,2021-06-26,H24,23.0,24,2021-06-26 23:59:59,2021-06-26 23:59:59
21428,4,8,28079004_8_8,2021-06-27,H24,10.0,24,2021-06-27 23:59:59,2021-06-27 23:59:59
21429,4,8,28079004_8_8,2021-06-28,H24,18.0,24,2021-06-28 23:59:59,2021-06-28 23:59:59
21430,4,8,28079004_8_8,2021-06-29,H24,42.0,24,2021-06-29 23:59:59,2021-06-29 23:59:59


In [97]:
pd.to_datetime(poc_air_index8_4['TIME'] )

0       2019-04-01 00:59:59
1       2019-04-02 00:59:59
2       2019-04-03 00:59:59
3       2019-04-04 00:59:59
4       2019-04-05 00:59:59
                ...        
21427   2021-06-26 23:59:59
21428   2021-06-27 23:59:59
21429   2021-06-28 23:59:59
21430   2021-06-29 23:59:59
21431   2021-06-30 23:59:59
Name: TIME, Length: 21432, dtype: datetime64[ns]

In [98]:
poc_air_index8_4['TIME']=pd.to_datetime(poc_air_index8_4['TIME'] )

In [99]:
poc_air_index8_4

,ESTACION,MAGNITUD,PUNTO_MUESTREO,DATE,variable,value,HORA,TIME,TIMESTAMP
0,4,8,28079004_8_8,2019-04-01,H01,17.0,1,2019-04-01 00:59:59,2019-04-01 0:59:59
1,4,8,28079004_8_8,2019-04-02,H01,22.0,1,2019-04-02 00:59:59,2019-04-02 0:59:59
2,4,8,28079004_8_8,2019-04-03,H01,33.0,1,2019-04-03 00:59:59,2019-04-03 0:59:59
3,4,8,28079004_8_8,2019-04-04,H01,20.0,1,2019-04-04 00:59:59,2019-04-04 0:59:59
4,4,8,28079004_8_8,2019-04-05,H01,22.0,1,2019-04-05 00:59:59,2019-04-05 0:59:59
...,...,...,...,...,...,...,...,...,...
21427,4,8,28079004_8_8,2021-06-26,H24,23.0,24,2021-06-26 23:59:59,2021-06-26 23:59:59
21428,4,8,28079004_8_8,2021-06-27,H24,10.0,24,2021-06-27 23:59:59,2021-06-27 23:59:59
21429,4,8,28079004_8_8,2021-06-28,H24,18.0,24,2021-06-28 23:59:59,2021-06-28 23:59:59
21430,4,8,28079004_8_8,2021-06-29,H24,42.0,24,2021-06-29 23:59:59,2021-06-29 23:59:59
